In [ ]:
# Copyright 2019 Google LLC
# Copyright 2025 Anonymized Authors

# Licensed under the Apache License, Version 2.0 (the "License"); 
# you may not use this file except in compliance with the License. 
# You may obtain a copy of the License at
# https://www.apache.org/licenses/LICENSE-2.0
"""
This notebook evaluates all combinations of probs0 and probs1 for the 
NAS-Bench-101. As there are both adjacency matrix and operation encoding, we 
display the performance of the different variations.

Requirements: 

-This notebook requires that tensorflow and numpy be installed within the 
Python environment you are running this script in. 

"""
import matplotlib.pyplot as plt
import numpy as np

import sys; sys.path.append('..')
from utils.nas_utils import load_nasbench, SpecOneHot, run_xevolution, greedy_selection, crossover
from utils.plotting import plot_all
from IPython.display import clear_output

nasb = load_nasbench()


In [ ]:
# here, we test out different modes for the guided mutation

def guided_mutation_mode(nasb, population, mode="00"):
    psize = len(population)
    
    # Convert the inputs to NumPy arrays (assuming population is a list of objects with 'flat' attribute)
    inputs = [x.to_flat() for x in population]
    
    # Convert list of lists/arrays into a 2D NumPy array
    mutation_inputs = np.array(inputs)
    
    # Sum across the first axis (population size), axis=0 corresponds to summing across rows (for each feature)
    summed = np.sum(mutation_inputs, axis=0)

    # Define the probabilities for mutation
    probs_mat_1 = summed[:SpecOneHot.flat_matrix] / psize  # Mass of ones for flat matrix
    probs_mat_1 /= np.sum(probs_mat_1)  # Normalize so the sum of probabilities is 1
    probs_mat_0 = 1 - probs_mat_1  # Mass of zeros for flat matrix
    probs_mat_0 /= np.sum(probs_mat_0)  # Normalize so the sum of probabilities is 1

    probs_ops_1 = summed[SpecOneHot.flat_matrix:] / psize  # Mass of ones for operations
    probs_ops_1 /= np.sum(probs_ops_1)  # Normalize so the sum of probabilities is 1
    probs_ops_0 = 1 - probs_ops_1
    probs_ops_0 /= np.sum(probs_ops_0)  # Normalize so the sum of probabilities is 1

    # Select which probabilities to use based on mode
    if mode=="00":
        probs_mat = probs_mat_0
        probs_ops = probs_ops_0
    elif mode=="01":
        probs_mat = probs_mat_0
        probs_ops = probs_ops_1
    elif mode=="10":
        probs_mat = probs_mat_1
        probs_ops = probs_ops_0
    elif mode=="11":
        probs_mat = probs_mat_1
        probs_ops = probs_ops_1
    else:
        print("mode can either be 0 or 1")


    children = []
    for i in range(psize):
        while True:
            child_spec = mutation_inputs[i, :].astype(int)

            # Select indices based on the probabilities using np.random.choice (similar to torch.multinomial)
            mat_index_1 = np.random.choice(len(probs_mat_1), 1, p=probs_mat)[0]
            mat_index_0 = np.random.choice(len(probs_mat_0), 1, p=probs_mat)[0]
            ops_index = np.random.choice(len(probs_ops), 1, p=probs_ops)[0] + SpecOneHot.flat_matrix

            # Modify the child_spec array based on the selected indices
            base = (ops_index - (ops_index % 3))  # Ensure base is aligned to 3 operations
            remove = [base, base + 1, base + 2]
            child_spec[remove] = 0
            child_spec[ops_index] = 1

            # set 0 to 1
            base2 = (mat_index_1 - (mat_index_1%2))
            remove2 = [base2, base2+1] 
            child_spec[remove2] = 0
            child_spec[base2] = 1

            # set 1 to 0
            base3 = (mat_index_0 - (mat_index_0%2))
            remove3 = [base3, base3+1] 
            child_spec[remove3] = 0
            child_spec[base3+1] = 1

            # Assuming SpecOneHot.spec_from_flat is a function that returns a spec from a flat representation
            spec = SpecOneHot.spec_from_flat(child_spec)
            
            if nasb.is_valid(spec):
                children.append(spec)
                break

    return children

In [ ]:
# comparing against greedy evolution and regularized evolution
gm00 = lambda nasb, crossed_parents: guided_mutation_mode(nasb, crossed_parents, "00")
gm01 = lambda nasb, crossed_parents: guided_mutation_mode(nasb, crossed_parents, "01")
gm10 = lambda nasb, crossed_parents: guided_mutation_mode(nasb, crossed_parents, "10")
gm11 = lambda nasb, crossed_parents: guided_mutation_mode(nasb, crossed_parents, "11")


exp1 = {
    "data": 
    {
        "gm00" : [[],"Light Blue"],
        "gm01" : [[],"Dark Blue"],
        "gm10" : [[],"Red Orange"],
        "gm11" : [[],"Dark Orange"],

    },
    "config": 
    {
        "budget" : int(5e6),
        "limits" : (0.9365, 0.9435),
        "n" : 100,
        "print_every" : 2,
        "confidence_intervall" : True,
        "pvalue" : 0.05,
        "significant_areas": False,
        "dataset" : "test",
   }
}

budget = exp1["config"]["budget"]
for run in range(exp1["config"]["n"]):
    nasb.reset_budget_counters()
    times, best_valid, best_test = run_xevolution(nasb, greedy_selection, crossover, gm00, budget, 50)
    exp1["data"]["gm00"][0].append((times, best_valid, best_test))

    nasb.reset_budget_counters()
    times, best_valid, best_test = run_xevolution(nasb, greedy_selection, crossover, gm01, budget, 50)
    exp1["data"]["gm01"][0].append((times, best_valid, best_test))

    nasb.reset_budget_counters()
    times, best_valid, best_test = run_xevolution(nasb, greedy_selection, crossover, gm10, budget, 50)
    exp1["data"]["gm10"][0].append((times, best_valid, best_test))

    nasb.reset_budget_counters()
    times, best_valid, best_test = run_xevolution(nasb, greedy_selection, crossover, gm11, budget, 50)
    exp1["data"]["gm11"][0].append((times, best_valid, best_test))


    if (run % exp1["config"]["print_every"] == 0):
        clear_output(wait=True)
        fig, ax = plt.subplots()
        plot_all(exp1, ax)
        plt.show()
        print('Running repeat %d' % (run + 1))

clear_output(wait=True)
fig, ax = plt.subplots()
plot_all(exp1, ax)
plt.show()